<a href="https://colab.research.google.com/github/viniciusgcjr/linkedin_posts/blob/main/el_nino_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

#Reading the data from NOAA website

In [5]:
col_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
df_oni = pd.read_csv('https://psl.noaa.gov/data/correlation/oni.data', sep='\s+', skiprows=1, names=col_names)
df_oni = df_oni.apply(pd.to_numeric, errors='coerce')
df_oni.dropna(inplace=True)
df_oni.replace({-99.90 : np.NaN}, inplace=True)
df_oni

,January,February,March,April,May,June,July,August,September,October,November,December
1950,-1.53,-0.67,-0.43,-0.14,0.14,-0.85,-0.54,-0.42,-0.39,-0.44,-0.60,-0.80
1951,-0.82,-0.54,-0.17,0.18,0.36,0.58,0.70,0.89,0.99,1.15,1.04,0.81
1952,0.53,0.37,0.34,0.29,0.20,0.00,-0.08,0.00,0.15,0.10,0.04,0.15
1953,0.40,0.60,0.63,0.66,0.75,0.77,0.75,0.73,0.78,0.84,0.84,0.81
1954,0.76,0.47,-0.05,-0.41,-0.54,-0.50,-0.64,-0.84,-0.90,-0.77,-0.73,-0.66
...,...,...,...,...,...,...,...,...,...,...,...,...
2019,0.75,0.72,0.71,0.66,0.54,0.45,0.28,0.14,0.19,0.35,0.51,0.55
2020,0.50,0.48,0.40,0.19,-0.08,-0.30,-0.41,-0.57,-0.89,-1.17,-1.27,-1.19
2021,-1.05,-0.93,-0.84,-0.66,-0.48,-0.38,-0.40,-0.49,-0.67,-0.81,-0.98,-0.98
2022,-0.97,-0.93,-0.99,-1.06,-0.99,-0.85,-0.81,-0.91,-1.01,-0.99,-0.92,-0.82


In [6]:
pd.options.plotting.backend = "plotly"
fig = df_oni.plot()
fig.show()

In [7]:
import plotly.express as px

In [8]:
fig = px.imshow(df_oni.T,
                color_continuous_scale='RdBu_r',
                title='Oceanic Niño Index',
                labels=dict(x="Year", y="Month", color="ONI"),
                aspect='auto')
fig.show()

In [9]:
fig = px.imshow(df_oni,
                color_continuous_scale='RdBu_r',
                title='Oceanic Niño Index',
                labels=dict(x="Year", y="Month", color="ONI"),
                aspect='auto')
fig.show()

# ONI SERIES

In [10]:
stacked_series = df_oni.stack()
stacked_series


1950  January    -1.53
      February   -0.67
      March      -0.43
      April      -0.14
      May         0.14
                  ... 
2022  December   -0.82
2023  January    -0.67
      February   -0.43
      March      -0.14
      April       0.14
Length: 880, dtype: float64

In [11]:
type(stacked_series)

pandas.core.series.Series

In [12]:
stacked_series.to_frame().index.names

FrozenList([None, None])

In [13]:
df_oni_stacked = stacked_series.to_frame()
df_oni_stacked.reset_index(inplace=True)
df_oni_stacked.columns = ['Year', 'Month', 'ONI']
df_oni_stacked

,Year,Month,ONI
0,1950,January,-1.53
1,1950,February,-0.67
2,1950,March,-0.43
3,1950,April,-0.14
4,1950,May,0.14
...,...,...,...
875,2022,December,-0.82
876,2023,January,-0.67
877,2023,February,-0.43
878,2023,March,-0.14


In [14]:
df_oni_stacked['Date'] = pd.to_datetime(df_oni_stacked['Year'] + '-' + df_oni_stacked['Month'])
df_oni_stacked['Date'] = df_oni_stacked['Date'].dt.strftime('%Y-%m')
df_oni_stacked.set_index('Date', inplace=True)
df_oni_stacked.drop(['Year', 'Month'], axis=1, inplace=True)
df_oni_stacked

,ONI
Date,
1950-01,-1.53
1950-02,-0.67
1950-03,-0.43
1950-04,-0.14
1950-05,0.14
...,...
2022-12,-0.82
2023-01,-0.67
2023-02,-0.43


In [15]:
fig = px.line(df_oni_stacked, facet_col_spacing = 0.013)
fig.show()

In [16]:
import pandas as pd
import plotly.express as px

# Create new columns for negative and positive values
df_oni_stacked['Positive'] = df_oni_stacked['ONI'].where(df_oni_stacked['ONI'] >= 0)
df_oni_stacked['Negative'] = df_oni_stacked['ONI'].where(df_oni_stacked['ONI'] < 0)


fig = px.bar(df_oni_stacked, x=df_oni_stacked.index, y=['Positive', 'Negative'])
fig.show()


## DECADES

In [17]:
df_oni_stacked.index = pd.DatetimeIndex(df_oni_stacked.index)
df_oni_stacked = df_oni_stacked.reset_index()
df_oni_stacked['Decade'] = (df_oni_stacked['Date'].dt.year // 10) * 10

fig = px.line(df_oni_stacked, x='Date', y='ONI', color='Decade')
fig.show()


### DECADES - Monthly Mean

In [18]:
df_oni_melted = df_oni.copy()

df_oni_melted.reset_index(inplace=True)
df_oni_melted = df_oni_melted.melt(id_vars='index', var_name='Month', value_name='Value')
df_oni_melted['Month'] = pd.to_datetime(df_oni_melted['Month'], format='%B').dt.month
df_oni_melted = df_oni_melted.rename(columns={'index': 'Year', 'Value':'ONI'})
df_oni_melted = df_oni_melted.sort_values(['Year', 'Month'])
df_oni_melted = df_oni_melted.reset_index().drop(['index'], axis=1)
df_oni_melted

,Year,Month,ONI
0,1950,1,-1.53
1,1950,2,-0.67
2,1950,3,-0.43
3,1950,4,-0.14
4,1950,5,0.14
...,...,...,...
883,2023,8,NaN
884,2023,9,NaN
885,2023,10,NaN
886,2023,11,NaN


In [19]:
df_oni_melted['Date'] = pd.to_datetime(df_oni_melted['Year'] + '-' + df_oni_melted['Month'].astype(str))
df_oni_melted['Date'] = pd.to_datetime(df_oni_melted['Date'].dt.strftime('%Y-%m'))
df_oni_melted['Decade'] = (df_oni_melted['Date'].dt.year // 10) * 10
df_oni_melted.drop(['Date', 'Year'], axis=1, inplace=True)
df_oni_melted


,Month,ONI,Decade
0,1,-1.53,1950
1,2,-0.67,1950
2,3,-0.43,1950
3,4,-0.14,1950
4,5,0.14,1950
...,...,...,...
883,8,NaN,2020
884,9,NaN,2020
885,10,NaN,2020
886,11,NaN,2020


In [20]:
# Calculate the mean value for each decade
df_oni_monthly = df_oni_melted.groupby(['Decade', 'Month'])['ONI'].mean()
df_oni_monthly = df_oni_monthly.to_frame()
df_oni_monthly.reset_index(inplace=True)

In [21]:
fig = px.line(df_oni_monthly, x='Month', y='ONI', color='Decade')
fig.show()